# Imports


In [1]:
%pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)
print(torch.version.cuda)

print(torch.cuda.is_available())

2.4.1+cu118
0.19.1+cu118
11.8
False


# **Data Load**


In [3]:
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/europarl", "en-uk")

README.md:   0%|          | 0.00/76.3k [00:00<?, ?B/s]

ValueError: BuilderConfig 'en-uk' not found. Available: ['bg-cs', 'bg-da', 'bg-de', 'bg-el', 'bg-en', 'bg-es', 'bg-et', 'bg-fi', 'bg-fr', 'bg-hu', 'bg-it', 'bg-lt', 'bg-lv', 'bg-nl', 'bg-pl', 'bg-pt', 'bg-ro', 'bg-sk', 'bg-sl', 'bg-sv', 'cs-da', 'cs-de', 'cs-el', 'cs-en', 'cs-es', 'cs-et', 'cs-fi', 'cs-fr', 'cs-hu', 'cs-it', 'cs-lt', 'cs-lv', 'cs-nl', 'cs-pl', 'cs-pt', 'cs-ro', 'cs-sk', 'cs-sl', 'cs-sv', 'da-de', 'da-el', 'da-en', 'da-es', 'da-et', 'da-fi', 'da-fr', 'da-hu', 'da-it', 'da-lt', 'da-lv', 'da-nl', 'da-pl', 'da-pt', 'da-ro', 'da-sk', 'da-sl', 'da-sv', 'de-el', 'de-en', 'de-es', 'de-et', 'de-fi', 'de-fr', 'de-hu', 'de-it', 'de-lt', 'de-lv', 'de-nl', 'de-pl', 'de-pt', 'de-ro', 'de-sk', 'de-sl', 'de-sv', 'el-en', 'el-es', 'el-et', 'el-fi', 'el-fr', 'el-hu', 'el-it', 'el-lt', 'el-lv', 'el-nl', 'el-pl', 'el-pt', 'el-ro', 'el-sk', 'el-sl', 'el-sv', 'en-es', 'en-et', 'en-fi', 'en-fr', 'en-hu', 'en-it', 'en-lt', 'en-lv', 'en-nl', 'en-pl', 'en-pt', 'en-ro', 'en-sk', 'en-sl', 'en-sv'

In [5]:
import spacy

spacy_en = spacy.load('en_core_web_sm')
spacy_uk = spacy.load('uk_core_news_sm')


Данные успешно загружены из локального файла.


In [6]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_uk(text):
    return [tok.text for tok in spacy_uk.tokenizer(text)]

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
<class 'pandas.core.frame.DataFrame'>
